In [1]:
import pandas as pd
import inspect
import os

class Layer:
    entities = []
    
    def __init__(self, project_name,environment):
        self.project_name = project_name
        self.environment = environment
        
    def setup(self):
        if os.path.exists(self.environment):
            file1 = open(self.environment, 'r')
            for lib in file1.readlines():
                print(f"Installing {lib.strip()}...")
        else:
            print(f"Environment file not found: {self.environment}")
    
    def log_metric(self, metric, value):
        parent_function = eval(inspect.stack()[1][3])
        entity = parent_function._name
        print(f"{entity} > {metric}:{value}")
    
    def run(self, entities):
        for entity in entities:
            if entity._type == "feature":
                self.entities.append(Feature(entity))
            elif entity._type == "model":
                self.entities.append(Model(entity))
            
        print(f"--- Running Project: {self.project_name} ---")
        
        self.setup()
        
        for entity in self.entities:
            entity.run()
        print(f"\n--- Run Complete! ---")
    
    def get_dataset(self,name):
        if name=="users":
            data = [[1,'tom', '01011999'], [2,'nick', '01011983'], [3,'juli', '01012002']]
            columns = ['id', 'name', 'birth']

            df = pd.DataFrame(data, columns = columns)
            return df
        else:
            for entity in self.entities:
                if entity.name == name:
                    return entity.result
            


class Model:
    result = None
    
    def __init__(self, func):
        self.name = func._name
        self.func = func
    def run(self):
        print(f"\nTraining {self.name}")
        self.result = self.func()

class Feature:
    result = None
    
    def __init__(self, func):
        self.name = func._name
        self.func = func
    def run(self):
        print(f"\nBuilding {self.name}")
        self.result = self.func()

class Context:
    def __init__(self):
        pass
    def log_metric(self, metric, value):
        print(f"Metric > {metric}:{value}")

def derived_dataset(name):
    def inner(func):
        func._type = "feature"
        func._name = name
        return func        
    return inner
 
def model(name):
    def inner(func):
        func._type = "model"
        func._name = name
        return func        
    return inner


In [5]:
@derived_dataset('users_dataset')
def build_users():
    df = layer.get_dataset("users")
    df['age'] = pd.to_datetime(df['birth'], format='%m%d%Y')
    df['age'] =  (pd.Timestamp('now') - df['age']).astype('<m8[Y]')
    layer.log_metric("Users Count", len(df))
    return df

@model('ltv_model')
def train():
    df = layer.get_dataset("users_dataset")
    print(df.head())
    accuracy = 0.87
    layer.log_metric("accuracy", accuracy)

# ++ init Layer
layer = Layer(project_name = "ltv_project", environment='requirements.txt')

# ++ To run the whole project on Layer Infra
# layer.run([build_users, train])

# ++ To train model on Layer infra
# layer.run([train])

# ++ To debug the code locally, just call the function:
build_users()

users_dataset > Users Count:3


,id,name,birth,age
0,1,tom,01011999,22.0
1,2,nick,01011983,38.0
2,3,juli,01012002,19.0
